Data Shift Exploration
===

Mocking labeled revisions so that rev-scoring can be applied.

#### Analysis plan:

Basic intuition:

For continuous variables, use the two-sample K-S test.
For binary variables, use the two-sample Chi Squared.  (Could consider a G-test instead...)

To compare source and target distribution, conduct a test for each variable independently.  Then, apply Bonf. correction for the multiple comparisons (Bonf. correction equal to the number of variables.)
Then, reject the null hypothesis if the minimum p-value among all tests is less than 0.05/K (K is number of tests).
Thus, ANY significant difference after correction is taken to be evidence of distribution shift.


How to measure magnitude?
This is a hard problem... Could sum the Earth Mover's Distance, perhaps?  But, would want to normalize by the range of the feature in some way.
I need to do more reading about how to quantify the magnitude of a data shift.
Could use multivariate KL divergence?

Could also use a Multivariate test:
Consider either Maximum Mean Discrepancy (as Lipton used) or Cramer's test: https://cran.r-project.org/web/packages/cramer/cramer.pdf

Should also train a classifier 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
import mwapi
import mwxml
import mwxml.utilities
import mwcli
import mwreverts
import oresapi
import mwparserfromhell

In [3]:
import os
import requests
from tqdm import tqdm
import bz2
import gzip
import json
import re
import hashlib
from datetime import datetime
import nltk
import scipy.stats
import para
from itertools import groupby
from collections import Counter
import time

In [4]:
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = git_root_dir[0]
git_root_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback'

In [5]:
revisions_features_filepath = os.path.join(git_root_dir, "data/raw/editquality/datasets/enwiki.labeled_revisions.20k_2015.damaging.tsv")
assert os.path.exists(revisions_features_filepath)

### Load training features

In [6]:
features_df = pd.read_csv(revisions_features_filepath, sep='\t', header=0)
len(features_df)

19348

In [7]:
rev_list = []
revisions_with_cache_filepath = os.path.join(git_root_dir, "data/raw/editquality/datasets/enwiki.labeled_revisions.w_cache.20k_2015.json")
with open(revisions_with_cache_filepath, 'r') as infile:
    for line in infile:
        rev = json.loads(line)
        rev_list.append({
            'rev_id': rev['rev_id'],
            'damaging': rev['damaging'],
            'goodfaith': rev['goodfaith']
        })
df = pd.DataFrame(rev_list)
len(df)

19348

In [8]:
df = pd.concat([df, features_df], axis=1)

In [9]:
# Load in the labeled revisions content data
labeled_revs_dir = os.path.join(git_root_dir, "data/derived/labeled-revs")
labeled_revs_filepath = os.path.join(labeled_revs_dir, 'labeled_revisions.20k_2015.content.ndjson')
results = []
with open(labeled_revs_filepath, 'r') as infile:
    for line in infile:
        result = json.loads(line)
        results.append(result)

#Identify revision ids for which content is available
rev_ids_with_content = set()
for result in results:
    for page in result['pages']:
        for rev in page['revisions']:
            rev_id = rev['revid']
            rev_ids_with_content.add(rev_id)
df['has_content'] = df.rev_id.map(lambda rev_id: rev_id in rev_ids_with_content)
f"{np.sum(df.has_content)} / {len(df)} ({np.sum(df.has_content) / len(df)*100:.2f}%) training revisions have associated content."

'19344 / 19348 (99.98%) training revisions have associated content.'

In [10]:
rev_ids_meeting_criteria = set()
for result in results:
    for page in result['pages']:
        if page['ns'] != 0 or 'redirect' in page:
            continue
        for rev in page['revisions']:
            rev_id = rev['revid']
            rev_ids_meeting_criteria.add(rev_id)
df['meets_criteria'] = df.rev_id.map(lambda rev_id: rev_id in rev_ids_meeting_criteria)
len(rev_ids_meeting_criteria)

13559

In [11]:
features_df = df.loc[df.meets_criteria,features_df.columns]
features_df.shape

(13529, 82)

### Load sample1 features

In [12]:
raw_data_dir = "/export/scratch2/wiki_data"
derived_data_dir = os.path.join(git_root_dir, "data", "derived")
mock_features_filepath = os.path.join(git_root_dir, "data/derived/labeled-revs/sample1.mock.damaging.tsv")

In [13]:
mock_features_df = pd.read_csv(mock_features_filepath, sep='\t', header=0)
len(mock_features_df)

458449

### Load month_sample features

In [14]:
month_sample_features_dir = "/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived/stub-history-all-revisions/month_sample/revscoring_features"
month_sample_filepath = os.path.join(month_sample_features_dir, "rev_ids_month_sample_2015_01.mock.damaging.tsv")

In [15]:
month_sample_df = pd.read_csv(month_sample_filepath, sep='\t', header=0)
len(month_sample_df)

99789

### Multiple Univariate Testing

In [54]:
# define source and target dataframes
sdf = features_df.drop(columns='damaging')
#tdf = month_sample_df.drop(columns='damaging')
tdf = mock_features_df.drop(columns='damaging')

In [55]:
sdf.shape, tdf.shape

((13529, 80), (458449, 80))

In [56]:
sdf.dtypes

feature.revision.page.is_articleish                                                   bool
feature.revision.page.is_mainspace                                                    bool
feature.revision.page.is_draftspace                                                   bool
feature.log((wikitext.revision.parent.chars + 1))                                  float64
feature.log((len(<datasource.tokenized(datasource.revision.parent.text)>) + 1))    float64
                                                                                    ...   
feature.english.dictionary.revision.diff.non_dict_word_delta_increase                int64
feature.english.dictionary.revision.diff.non_dict_word_delta_decrease                int64
feature.english.dictionary.revision.diff.non_dict_word_prop_delta_sum              float64
feature.english.dictionary.revision.diff.non_dict_word_prop_delta_increase         float64
feature.english.dictionary.revision.diff.non_dict_word_prop_delta_decrease         float64

In [57]:
n_features = len(sdf.columns)
univariate_results = []
for col_name, col_dtype in zip(sdf.columns, sdf.dtypes):
    is_feature_binary = False
    std = 0
    if col_dtype == bool:
        is_feature_binary  = True
        s_true_count = np.sum(sdf[col_name])
        s_false_count = len(sdf) - s_true_count
        t_true_count = np.sum(tdf[col_name])
        t_false_count = len(tdf) - t_true_count
        if s_true_count == 0 or s_false_count == 0 or t_true_count == 0 or t_false_count == 0:
            p = n_features
            diff = (t_true_count / len(tdf)) - (s_true_count / len(sdf))
        else:
            cont = np.array(
                [[s_true_count, s_false_count],
                 [t_true_count, t_false_count]]
            )
            chi2, p, dof, expctd = scipy.stats.chi2_contingency(cont)
            p = p * n_features
        diff = (t_true_count / len(tdf)) - (s_true_count / len(sdf))
    else:
        D, p = scipy.stats.ks_2samp(sdf[col_name], tdf[col_name])
        p = p * n_features
        diff = np.mean(tdf[col_name]) - np.mean(sdf[col_name])
        std = np.std(np.concatenate((sdf[col_name], tdf[col_name])))
    pre_mean = np.mean(sdf[col_name])
    _, mean_diff_p = scipy.stats.ttest_ind(sdf[col_name], tdf[col_name], equal_var=False)
    mean_diff_p *= n_features
    tup = (col_name, is_feature_binary, p, diff, std, pre_mean, diff / pre_mean, mean_diff_p)
    univariate_results.append(tup)
univariate_results.sort(key=lambda tup: (tup[1], abs(tup[2]), -abs(tup[6])), reverse=False)
print(f"{'Feature Name':<71} {'Feat. Type':>10} {'p-value':>7} {'S'} {'T'} {'Diff':>6}")
print("="*100)
for tup in univariate_results:
    col_name, is_feature_binary, p, diff, std, pre_mean, pct_change, mean_diff_p = tup
    print(f"{col_name[8:]:<71} {'binary' if is_feature_binary else 'continuous':>10} {p:7.3f} {'Y' if p < 0.001 else '-'} {'Y' if mean_diff_p < 0.001 else '-'} {diff:6.2f} ({pre_mean:.2f}) {pct_change*100:.1f}%")


/export/scratch2/levon003/bin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars


Feature Name                                                            Feat. Type p-value S T   Diff
log((wikitext.revision.parent.ref_tags + 1))                            continuous   0.000 Y Y   0.92 (2.32) 39.7%
log((wikitext.revision.parent.external_links + 1))                      continuous   0.000 Y Y   0.87 (2.28) 38.1%
log((wikitext.revision.parent.templates + 1))                           continuous   0.000 Y Y   0.98 (3.02) 32.4%
log((wikitext.revision.parent.headings + 1))                            continuous   0.000 Y Y   0.47 (2.11) 22.2%
log((wikitext.revision.parent.wikilinks + 1))                           continuous   0.000 Y Y   0.73 (4.12) 17.7%
log((len(<datasource.wikitext.revision.parent.words>) + 1))             continuous   0.000 Y Y   0.90 (6.97) 12.9%
log((len(<datasource.tokenized(datasource.revision.parent.text)>) + 1)) continuous   0.000 Y Y   0.93 (8.01) 11.6%
log((wikitext.revision.parent.chars + 1))                               continuous   0.000 Y 

### Multivariate testing

Multivariate kernel two-sample tests

To get a p-value directly comparing the two samples, using Maximum Mean Discrepancy or some other test.

TODO Do this in R lol, no fast Python implementation exists as far as I can tell.

### Compute level of difference

In [23]:
def KLdivergence(x, y):
    """Compute the Kullback-Leibler divergence between two multivariate samples.
    Parameters
    ----------
    x : 2D array (n,d)
    Samples from distribution P, which typically represents the true
    distribution.
    y : 2D array (m,d)
    Samples from distribution Q, which typically represents the approximate
    distribution.
    Returns
    -------
    out : float
    The estimated Kullback-Leibler divergence D(P||Q).
    References
    ----------
    Pérez-Cruz, F. Kullback-Leibler divergence estimation of
    continuous distributions IEEE International Symposium on Information
    Theory, 2008.
    
    https://gist.github.com/atabakd/ed0f7581f8510c8587bc2f41a094b518
    """
    from scipy.spatial import cKDTree as KDTree

    # Check the dimensions are consistent
    x = np.atleast_2d(x)
    y = np.atleast_2d(y)

    n,d = x.shape
    m,dy = y.shape

    assert(d == dy)


    # Build a KD tree representation of the samples and find the nearest neighbour
    # of each point in x.
    xtree = KDTree(x)
    ytree = KDTree(y)

    # Get the first two nearest neighbours for x, since the closest one is the
    # sample itself.
    r = xtree.query(x, k=2, eps=.01, p=2)[0][:,1]
    s = ytree.query(x, k=1, eps=.01, p=2)[0]

    # There is a mistake in the paper. In Eq. 14, the right side misses a negative sign
    # on the first term of the right hand side.
    return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))

In [24]:
sdf.to_numpy().shape

(13529, 80)

In [52]:
# KL divergence comparing the training data to the April 2015 data
tdf = month_sample_df.drop(columns='damaging')
start = datetime.now()
kl_sample_n = 10000
n_iters = 10
kld_list = []
for i in range(n_iters):
    kld = KLdivergence(sdf.sample(n=kl_sample_n).to_numpy(), (tdf.sample(n=kl_sample_n) + 0.0000000001).to_numpy())
    kld_list.append(kld)
print(f"{datetime.now() - start}")
print(f"{np.mean(kld_list):.2f} w std. {np.std(kld_list):.2f}")

0:03:47.998024
2.02 w std. 0.50


In [53]:
# KL divergence comparing the training data to the sample1 data
tdf = mock_features_df.drop(columns='damaging')
start = datetime.now()
kl_sample_n = 10000
n_iters = 10
kld_list = []
for i in range(n_iters):
    kld = KLdivergence(sdf.sample(n=kl_sample_n).to_numpy(), (tdf.sample(n=kl_sample_n) + 0.0000000001).to_numpy())
    kld_list.append(kld)
print(f"{datetime.now() - start}")
print(f"{np.mean(kld_list):.2f} w std. {np.std(kld_list):.2f}")

0:04:16.450829
6.37 w std. 0.31
